In [1]:
import torchvision.transforms as tt
import PIL.Image as Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
import datasets.GeneralDataset as GenDataset
from san_vision import transforms

In [1]:

import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter

resnet18 = models.resnet18(False)
writer = SummaryWriter()
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]
writer.add_graph(resnet18)
for n_iter in range(100):

    dummy_s1 = torch.rand(1)
    dummy_s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar('data/scalar1', dummy_s1[0], n_iter)
    writer.add_scalar('data/scalar2', dummy_s2[0], n_iter)

    writer.add_scalars('data/scalar_group', {'xsinx': n_iter * np.sin(n_iter),
                                             'xcosx': n_iter * np.cos(n_iter),
                                             'arctanx': np.arctan(n_iter)}, n_iter)

    dummy_img = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)

        dummy_audio = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # amplitude of sound should in [-1, 1]
            dummy_audio[i] = np.cos(freqs[n_iter // 10] * np.pi * float(i) / float(sample_rate))
        writer.add_audio('myAudio', dummy_audio, n_iter, sample_rate=sample_rate)

        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)

        for name, param in resnet18.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), n_iter)

        # needs tensorboard 0.4RC or later
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(100), n_iter)

dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]

features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

# export scalar data to JSON for external processing
writer.export_scalars_to_json("./all_scalars.json")
writer.close()

Error occurs, No graph saved


TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not NoneType

In [2]:
imp = '/home/abhirup/Datasets/300W-Style/300W-Original/afw/59354095_3.jpg'

In [3]:
img = cv2.imread(imp)

In [4]:
def to_ten(pics):
    """
    Args:
      pic (PIL.Image or numpy.ndarray): Image to be converted to tensor.
      points 3 * N numpy.ndarray [x, y, visiable] or Point_Meta
    Returns:
      Tensor: Converted image.
    """
    ## add to support list
    if isinstance(pics, list): is_list = True
    else:                      is_list, pics = False, [pics]

    returned = []
    for pic in pics:
        if isinstance(pic, np.ndarray):
        # handle numpy array
           img = torch.from_numpy(pic.transpose((2, 0, 1)))
        # backward compatibility
           returned.append( img.float().div(255) )
        continue

      # handle PIL Image
        if pic.mode == 'I':
            img = torch.from_numpy(np.array(pic, np.int32, copy=False))
        elif pic.mode == 'I;16':
            img = torch.from_numpy(np.array(pic, np.int16, copy=False))
        else:
            img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
      # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK
        if pic.mode == 'YCbCr':
            nchannel = 3
        elif pic.mode == 'I;16':
            nchannel = 1
        else:
            nchannel = len(pic.mode)
        img = img.view(pic.size[1], pic.size[0], nchannel)
      # put it from HWC to CHW format
      # yikes, this transpose takes 80% of the loading time/CPU
        img = img.transpose(0, 1).transpose(0, 2).contiguous()
        if isinstance(img, torch.ByteTensor):
            img = img.float().div(255)
        returned.append(img)

    if is_list == False:
        assert len(returned) == 1, 'For non-list data, length of answer must be one not {}'.format(len(returned))
        returned = returned[0]

    return returned
def to_pil(pic):
    """
    Args:
      pic (Tensor or numpy.ndarray): Image to be converted to PIL.Image.
    Returns:
      PIL.Image: Image converted to PIL.Image.
    """
    npimg = pic
    mode = None
    if isinstance(pic, torch.FloatTensor):
        pic = pic.mul(255).byte()
    if torch.is_tensor(pic):
        npimg = np.transpose(pic.numpy(), (1, 2, 0))
    assert isinstance(npimg, np.ndarray), 'pic should be Tensor or ndarray'
    if npimg.shape[2] == 1:
        npimg = npimg[:, :, 0]

        if npimg.dtype == np.uint8:
            mode = 'L'
        if npimg.dtype == np.int16:
            mode = 'I;16'
        if npimg.dtype == np.int32:
            mode = 'I'
        elif npimg.dtype == np.float32:
            mode = 'F'
    else:
        if npimg.dtype == np.uint8:
            mode = 'RGB'
    assert mode is not None, '{} is not supported'.format(npimg.dtype)
    return Image.fromarray(npimg, mode=mode)


In [5]:
Ten = tt.ToTensor()
Im = tt.ToPILImage()

In [6]:
ten = Ten(img)
ten2 = to_ten(img)

In [7]:
mean_fill   = tuple( [int(x*255) for x in [0.5, 0.5, 0.5] ] )
normalize   = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                      std=[0.5, 0.5, 0.5])
train_transform = [transforms.PreCrop(0.2)]
train_transform = [transforms.TrainScale2WH((256, 256))]
train_transform += [transforms.AugScale(1.1, 1, 1)]
train_transform += [transforms.AugCrop(256, 256, 30, mean_fill)]
train_transform += [transforms.ToTensor()]
train_transform = transforms.Compose( train_transform )

In [8]:
train_list = '/home/abhirup/Datasets/300W-Style/box-coords/300W-Original/300w.train.GTB'
train_data = GenDataset(train_transform, 4, 8, 'gaussian', 'im_test')
train_data.load_list(train_list, 68, True)

The general dataset initialization done, sigma is 4, downsample is 8, dataset-name : im_test, self is : GeneralDataset(number of point=-1, heatmap_type=gaussian)
Load list from /home/abhirup/Datasets/300W-Style/box-coords/300W-Original/300w.train.GTB
Load [0/1]-th list : /home/abhirup/Datasets/300W-Style/box-coords/300W-Original/300w.train.GTB with 3148 images
Start load data for the general datas
Load data done for the general dataset, which has 3148 images.


In [11]:
eval_list = '/home/abhirup/Datasets/300W-Style/box-coords/300W-Original/test'
eval_transform  = transforms.Compose([transforms.PreCrop(0.2),
                                      transforms.TrainScale2WH((256, 256)),
                                      transforms.ToTensor(), normalize])
eval_data = GenDataset(eval_transform, 4, 8, 'gaussian', 'test')
eval_loader = torch.utils.data.DataLoader(eval_data, batch_size=5, shuffle=False, pin_memory=False)

The general dataset initialization done, sigma is 4, downsample is 8, dataset-name : test, self is : GeneralDataset(number of point=-1, heatmap_type=gaussian)
